# Seq2seq RNN using Keras

<p style="text-align:justify;">This project is simply a walk through the code I found on the [Keras's blog](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html). The goal is to use LSTM to translate a sentence from English to French. The difficulty of this assignement is the sizes of the input and outpu data. Indeed, we have various unknowned sizes for the input sentences and the output is not likely to have the same size as the input.</p>

<div class='alert alert-info'>"the cat sat on the mat" -> [Seq2Seq model] -> "le chat etait assis sur le tapis"
</div>

## Loading the datas

In [9]:
from __future__ import print_function

from keras.models import Model, load_model
from keras.layers import LSTM, Dense, Input
import numpy as np

In [10]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = 'datas/fra.txt'

We start reading the datas and putting it inside numpy 3 dimensionnal vector of dimension (m, $T_{x}$, nbr_chars).

m: The number of examples<br>
$T_{x}$: The number of characters in the sentence. In this case, it is the nbr of chars in the longest sentence<br>
nbr_chars: The number of characters in the dictionnary for each language

<p style="text-align:justify;">It is interesting to note that we added \t at the beginning of each output datas and \n at the end of each output datas. The goal is to tell the algorithm where to start and where to stop. \t is given as the first character, then the RNN needs to predict the next one until \n.</p>

In [11]:
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for line in lines[:min(num_samples, len(lines)-1)]:
    input_text, target_text = line.split("\t")
    target_text = "\t"+target_text+"\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [12]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [13]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

<p style="text-align:justify;">The decoder output data is at T+1 compare to decoder input data. This is made like that so the RNN learns to predict the next character given the context of the encoder RNN.</p>

In [14]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1

## RNN Model

<p style="text-align:justify;">RNN models need a lot of computation power to be trained. In this example, running 100 epochs on a 10000 sentences dataset took 20 seconds for each epochs. The total computation took around 33 minutes on a computer using only the CPU.</p>

In [15]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.9210 - val_loss: 0.9688
Epoch 2/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.7362 - val_loss: 0.8026
Epoch 3/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.6259 - val_loss: 0.7149
Epoch 4/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.5692 - val_loss: 0.6640
Epoch 5/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.5300 - val_loss: 0.6273
Epoch 6/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.4984 - val_loss: 0.6058
Epoch 7/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.4708 - val_loss: 0.5784
Epoch 8/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.4485 - val_loss: 0.5529
Epoch 9/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.4276 - val_loss: 0.5435
Epoch 10/100
800

8000/8000 [==============================] - 21s 3ms/step - loss: 0.0801 - val_loss: 0.6969
Epoch 79/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.0794 - val_loss: 0.7001
Epoch 80/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.0779 - val_loss: 0.7092
Epoch 81/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.0767 - val_loss: 0.7061
Epoch 82/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.0757 - val_loss: 0.7130
Epoch 83/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.0742 - val_loss: 0.7193
Epoch 84/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.0736 - val_loss: 0.7207
Epoch 85/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.0725 - val_loss: 0.7252
Epoch 86/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.0713 - val_loss: 0.7240
Epoch 87/100
8000/8000 [==============================] - 21s 3ms/st

C:\Users\cnqv2027\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


## The inference mode

Here is how it is going to work:
1. Encode input and retrieve initial decoder state
2. Run one step of decoder with this initial state
   and a "start of sequence" token as target.
   Output will be the next target token
3. Repeat with the current target token and current states

In [16]:
#define sampling model
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [17]:
reverse_input_char_index = dict([(i, char) for i, char in enumerate(input_characters)])
reverse_target_char_index = dict([(i, char) for i, char in enumerate(target_characters)])

In [18]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1,num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1
    
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1
        
        states_value = [h, c]
    
    return decoded_sentence

In [19]:
for seq_index in range(100):
    #seq: seq+1 to keep the shape of (1, 16, 71) not (16, 71)
    input_seq = encoder_input_data[seq_index: seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: I see.
Decoded sentence: Je cross.

-
Input sentence: I try.
Decoded sentence: J'essaye.

-
Input sentence: I won!
Decoded sentence: Je l'ai emporté !

-
Input sentence: I won!
Decoded sentence: Je l'ai emporté !

-
Input sentence: Oh